# Introduction To Bolt

Hello! This is an intro to _Bolt_ to help you understand the structure of the framework. This way you'll hit the hit the ground running when you want to introduce models of your own. :)

``Bolt`` is a solver framework for kinetic theories and can be used to obtain solution for any equation of the following forms:

Conservative:
\begin{align*}
\frac{\partial f}{\partial t} + \frac{\partial (C_{q1} f)}{\partial q_1} + \frac{\partial (C_{q2} f)}{\partial q_2} +  \frac{\partial (C_{p1} f)}{\partial p_1} + \frac{\partial (C_{p2} f)}{\partial p_2} + \frac{\partial (C_{p3} f)}{\partial p_3} = S(f)
\end{align*}

Non-Conservative:
\begin{align*}
\frac{\partial f}{\partial t} + A_{q1} \frac{\partial f}{\partial q_1} + A_{q2} \frac{\partial f}{\partial q_2} + A_{p1} \frac{\partial f}{\partial p_1} + A_{p2} \frac{\partial f}{\partial p_2} + A_{p3} \frac{\partial f}{\partial p_3} = S(f)
\end{align*}

Where $A_{q1}$, $A_{q2}$, $A_{p1}$, $A_{p2}$, $A_{p3}$,$C_{q1}$, $C_{q2}$, $C_{p1}$, $C_{p2}$, $C_{p3}$ and $S(f)$  are terms that need to be coded in by the user. `Bolt` can make use of the advective semi-lagrangian method and/or the finite-volume method. While the advective semi-lagrangian method makes use of $A_{q1, q2, p1, p2, p3}$, while the finite volume method makes use of $C_{q1, q2, p1, p2, p3}$.

In this tutorial we'll be considering non-relativistic Boltzmann equation, with the BGK collision operator:

\begin{align*}
\frac{\partial f}{\partial t} + v_x \frac{\partial f}{\partial x} + v_y \frac{\partial f}{\partial y} + \frac{q}{m}(\vec{E} + \vec{v} \times \vec{B})_x \frac{\partial f}{\partial v_x} + \frac{q}{m}(\vec{E} + \vec{v} \times \vec{B})_y \frac{\partial f}{\partial v_y} + \frac{q}{m}(\vec{E} + \vec{v} \times \vec{B})_z \frac{\partial f}{\partial v_z} = C[f] = -\frac{f - f_0}{\tau}
\end{align*}

So for this model, we have the following:

$A_{q1} = C_{q1} = v_x$

$A_{q2} = C_{q2} = v_y$

$A_{p1} = C_{p1} = \frac{q}{m}(\vec{E} + \vec{v} \times \vec{B})_x$

$A_{p2} = C_{p2} = \frac{q}{m}(\vec{E} + \vec{v} \times \vec{B})_y$

$A_{p3} = C_{p3} = \frac{q}{m}(\vec{E} + \vec{v} \times \vec{B})_z$

$S(f) = -\frac{f-f_0}{\tau}$, where $f_0$ is the local-maxwellian distribution $\tau$ is the collison timescale.

Additionally, we've taken the generalized the canonical coordinates $p_1$, $p_2$ and $p_3$ to be the velocity values $v_x$, $v_y$, $v_z$. That is:

$p_1 = v_x$

$p_2 = v_y$

$p_3 = v_z$

Before we dive into how we introduce this non-relativistic Boltzmann equation into ``Bolt``, let's define the example problem that we intend to solve. The problem that we are considering is a simple one: Given an initial perturbation in the number density $n$ in a collisionless periodic 1D domain, how would the amplitude of density vary with time. Basically, we are stating that:

\begin{align*}
n(x, 0) = n_{background} + \delta n_r cos(kx) - \delta n_i sin(kx)
\end{align*}

Now ``Bolt`` requires the initial distribution function to be defined which we'll initialize using the Maxwell Boltzmann distribution function. The system that we are modelling here is a 1D1V one. That is, one dimension in position space, and one dimension in velocity space. The initial distribution function would be:

\begin{align*}
f(x, v, t = 0) = n(x, 0) \sqrt{\frac{m}{2 \pi k T}} e^{-\frac{mv^2}{2 k T}}
\end{align*}

The folder in which this notebook is contained has 4 other python files:``boundary_conditions.py``, ``domain.py``, ``initialize.py``, ``params.py`` each of which hold essential information about the system being modelled.

These files when which used with the ``import`` statement are imported as modules. These modules are what we use to use to pass the information to the solvers in ``Bolt``. We'll go ahead and import these modules for now, and explore what each of these files contain step by step.

In [3]:
# Importing problem specific modules:
import boundary_conditions
import domain
import params
import initialize

In [10]:
!cat boundary_conditions.py

in_q1_left  = 'periodic'
in_q1_right = 'periodic'

in_q2_bottom = 'periodic'
in_q2_top    = 'periodic'


As the name suggests ``boundary_conditions.py`` contains the information about the boundary conditions for the setup considered. While the current problem in consideration is for periodic boundary conditions, ``Bolt`` also supports Dirichlet, mirror, and shearing box boundary conditions. The setups for these boundary conditions can be found in other example problems. 

In [11]:
!cat domain.py

q1_start = 0
q1_end   = 1
N_q1     = 32

q2_start = 0
q2_end   = 1
N_q2     = 3

p1_start = -10
p1_end   = 10
N_p1     = 32

p2_start = -0.5
p2_end   = 0.5
N_p2     = 1

p3_start = -0.5
p3_end   = 0.5
N_p3     = 1
    
N_ghost_q = 3
N_ghost_p = 0


``domain.py`` contains data about the phase space domain and resolution that has been considered. Note that we've taken the number of grid points along ``q2`` as 3 although it's a 1D problem. It must be ensured that the number of grid zones in ``q1`` and ``q2`` are greater than or equal to the the number of ghost zones that are taken in q space. This is due to an internal restriction placed on us by one of the libraries we use for parallelization. Additionally, we've taken the domain zones and sizes for ``p3`` and ``p3`` such that ``dp2`` and ``dp3`` come out to be one. This way the integral measure, ``dp1 dp2 dp3`` boils down to be ``dp1`` which is then used for moment computations.

In [12]:
!cat params.py

import numpy as np
import arrayfire as af

fields_type       = 'electrostatic'
fields_initialize = 'fft'
fields_solver     = 'fdtd'

solver_method_in_q = 'FVM'
solver_method_in_p = 'FVM'

reconstruction_method_in_q = 'weno5'
reconstruction_method_in_p = 'weno5'

riemann_solver_in_q = 'upwind-flux'
riemann_solver_in_p = 'upwind-flux'

# Dimensionality considered in velocity space:
p_dim = 1

# Number of devices(GPUs/Accelerators) on each node:
num_devices = 1

# Constants:
mass               = [1]
boltzmann_constant = 1
charge             = [-10]

EM_fields_enabled = False
source_enabled    = True

# Variation of collisional-timescale parameter through phase space:
@af.broadcast
def tau(q1, q2, p1, p2, p3):
    return (0.01 * p1**0 * q1**0)

# Initial Conditions used in initialize:
rho_background         = 1
temperature_background = 1

p1_bulk_background = 0

pert_real = 0.01
pert_imag = 0.02

k_q1 = 2 * np.pi


Let's go over each of the attributes mentioned above to understand their usage:

``fields_type`` is used to declare what sort of fields are being solved in the problem of consideration. It can be electrostatic where magnetic fields stay at zero, electrodynamic where magnetic fields are also evolved and user-defined where the evolution of the fields in time are declared by the user(this is primarily used in debugging). This attribute can be set appropriately as ``electrostatic``, ``electrodynamic`` and ``user-defined``. The setup we've considered is an electrostatic one.

``fields_initialize`` is used to declare which method is used to set the initialize the values for the electromagnetic fields. 2 methods are available for initializing electrostatic fields from the density - ``snes`` and ``fft``. The ``fft`` method of initialization can only be used for serial runs with periodic boundary conditions. ``snes`` is  a more versatile method capable of being run in parallel with other boundary conditions as well. It makes use of the SNES routines in PETSc which use Krylov subspace methods to solve for the fields. Additionally, this can also be set to ``user-defined``, where the initial conditions for the electric and magnetic fields are defined in terms of ``q1`` and ``q2`` under initialize.

``fields_solver`` is used to declare which method is used to set the method that is used to evolve the fields with time. The same methods are available for computing electrostatic fields ``snes`` and ``fft``. The ``fdtd`` method is to be used when solving for electrodynamic systems.

``solver_method_in_q`` and ``solver_method_in_p`` are used to set the specific solver method used in q-space and p-space which can be set to ``FVM`` or ``ASL`` for the finite volume method and the advective semi-lagrangian method.

``reconstruction_method_in_q`` and ``reconstruction_method_in_p`` are used to set the specific reconstruction scheme used for the FVM method in q-space and p-space. This can be set to ``piecewise-constant``, ``minmod``, ``ppm`` and ``weno5``. 

``riemann_solver_in_q`` and ``riemann_solver_in_p`` are used to set the specific riemann solver which is to be used for the FVM method in q-space and p-space. This can be set to ``upwind-flux`` for the first order upwind flux method, and ``lax-friedrichs`` for the local Lax-Friedrichs method.

``p_dim`` is used to set the dimensionality considered in p-space. This becomes important especially for the definition of various physical quantities which vary as a function of the dimensionality and the moments. This would also be used in our case in the collision operator as we'll discuss further in this tutorial.

``num_devices`` is used in parallel runs when run on nodes which contain more than a single accelerator. For instance when running on nodes which contain 4 GPUs each, this attribute is set to 4.

``mass``, ``boltzmann_constant`` and ``charge`` are pretty explanatory from their name, and are used within certain methods of the solver.

``EM_fields_enabled`` is a solver flag which is used to indicate whether the case considered is one where we solve in p-space as well. Similarly, ``source_enabled`` is used to switch on and off the source term $S(f)$

``tau`` is the collision timescale in the BGK operator and used in solving for the source part of the equation.

The remaining parameters as they are mentioned are used the ``initialize`` module

In [13]:
!cat initialize.py

"""
Functions which are used in assigning the I.C's to
the system.
"""

import arrayfire as af
import numpy as np

def initialize_f(q1, q2, p1, p2, p3, params):

    m = params.mass
    k = params.boltzmann_constant

    rho_b = params.rho_background
    T_b   = params.temperature_background

    p1_bulk = params.p1_bulk_background

    pert_real = params.pert_real
    pert_imag = params.pert_imag

    k_q1 = params.k_q1

    # Calculating the perturbed density:
    rho = rho_b + (  pert_real * af.cos(k_q1 * q1)
                   - pert_imag * af.sin(k_q1 * q1)
                  )

    f = rho * (m / (2 * np.pi * k * T_b))**(1 / 2) \
            * af.exp(-m * (p1 - p1_bulk)**2 / (2 * k * T_b)) \

    af.eval(f)
    return (f)


As you can see the ``initialize`` module contains the function ``initialize_f`` which initializes the distribution function using the parameters that were declared.

## How the equation to be modelled is introduced into Bolt:

As one navigates from the root folder of this repository into the folder main folder for the package ``bolt``, there's two separate subfolders ``lib`` and ``src``. While all the files in ``lib`` contain the solver algorithms, and the structure for the solvers, ``src`` is where we introduce the models that we intend to model. For instance, the files that we'll be using for this test problem can be found under ``bolt/src/nonrelativistic_boltzmann``.

Let's start of by seeing how we'd introduce the above advection terms specific to the non-relativistic Boltzmann equation into the framework of ``Bolt``. Advection terms are introduced into ``Bolt`` by creating a module ``advection_terms`` which has the functions ``A_q, C_q, C_p, C_p``. 

It is expected that ``A_q`` and ``C_q`` take the arguments ``(f, t, q1, q2, v1, v2, v3, params)``, where `f` is the distribution function, `t` is the time elapsed, `(q1, q2, v1, v2, v3)` are phase space grid data for the position space and velocity space respectively. Additionally, it also accepts a module ``params`` which can take contain user defined attributes and functions that can be injected into this function.

While ``A_p`` and ``C_p`` take all the arguments that are taken by ``A_q`` and ``C_q``, it also takes the additional argument of a ``fields_solver`` object. The ``get_fields()`` method of this object returns the electromagnetic fields at the current instance. The ``fields_solver`` objects are internal to the solvers, and can be chosen as an electrostatic and electrodynamic 

In [2]:
# NOTE: NOT YET COMPLETED

ERROR:root:Line magic function `%paste` not found.


In [1]:
# Importing dependencies:
import arrayfire as af
import numpy as np
import pylab as pl
%matplotlib inline

In [2]:
from bolt.lib.physical_system import physical_system
from bolt.lib.nonlinear.nonlinear_solver import nonlinear_solver
from bolt.lib.linear.linear_solver import linear_solver

In [3]:
# Optimized plot parameters to make beautiful plots:
pl.rcParams['figure.figsize']  = 12, 7.5
pl.rcParams['figure.dpi']      = 300
pl.rcParams['image.cmap']      = 'jet'
pl.rcParams['lines.linewidth'] = 1.5
pl.rcParams['font.family']     = 'serif'
pl.rcParams['font.weight']     = 'bold'
pl.rcParams['font.size']       = 20
pl.rcParams['font.sans-serif'] = 'serif'
pl.rcParams['text.usetex']     = True
pl.rcParams['axes.linewidth']  = 1.5
pl.rcParams['axes.titlesize']  = 'medium'
pl.rcParams['axes.labelsize']  = 'medium'

pl.rcParams['xtick.major.size'] = 8
pl.rcParams['xtick.minor.size'] = 4
pl.rcParams['xtick.major.pad']  = 8
pl.rcParams['xtick.minor.pad']  = 8
pl.rcParams['xtick.color']      = 'k'
pl.rcParams['xtick.labelsize']  = 'medium'
pl.rcParams['xtick.direction']  = 'in'

pl.rcParams['ytick.major.size'] = 8
pl.rcParams['ytick.minor.size'] = 4
pl.rcParams['ytick.major.pad']  = 8
pl.rcParams['ytick.minor.pad']  = 8
pl.rcParams['ytick.color']      = 'k'
pl.rcParams['ytick.labelsize']  = 'medium'
pl.rcParams['ytick.direction']  = 'in'

In [4]:
# Defining the physical system to be solved:
system = physical_system(domain,
                         boundary_conditions,
                         params,
                         initialize,
                         advection_terms,
                         collision_operator.BGK,
                         moments
                        )

N_g_q = system.N_ghost_q

# Declaring a linear system object which will evolve the defined physical system:
nls = nonlinear_solver(system)
ls  = linear_solver(system)

NameError: name 'domain' is not defined

In [5]:
# Time parameters:
dt      = 0.001
t_final = 0.5

time_array  = np.arange(0, t_final + dt, dt)

rho_data_nls = np.zeros(time_array.size)
rho_data_ls  = np.zeros(time_array.size)

In [6]:
# Storing data at time t = 0:
n_nls           = nls.compute_moments('density')
rho_data_nls[0] = af.max(n_nls[:, :, N_g_q:-N_g_q, N_g_q:-N_g_q])

n_ls           = ls.compute_moments('density')
rho_data_ls[0] = af.max(n_ls) 

for time_index, t0 in enumerate(time_array[1:]):

    nls.strang_timestep(dt)
    ls.RK4_timestep(dt)

    n_nls                         = nls.compute_moments('density')
    rho_data_nls[time_index + 1]  = af.max(n_nls[:, :, N_g_q:-N_g_q, N_g_q:-N_g_q])
    
    n_ls                        = ls.compute_moments('density')
    rho_data_ls[time_index + 1] = af.max(n_ls) 

NameError: name 'nls' is not defined

In [7]:
pl.plot(time_array, rho_data_ls, '--', color = 'black', label = 'Linear Solver')
pl.plot(time_array, rho_data_nls, label='Nonlinear Solver')
pl.ylabel(r'MAX($\rho$)')
pl.xlabel('Time')
pl.legend()
pl.savefig('rho.png')
pl.clf()